# This script is mimicking what the backend would be doing when running an UDF

In [7]:
from os import getcwd
getcwd()

'/home/pzellner@eurac.edu/git_projects/r4openeo-usecases/uc2-ts-breakdetection/local_udf'

In [17]:
from openeo_r_udf.udf_lib import prepare_udf, execute_udf
import time
import xarray as xr

In [21]:
# config
load_file = "../00_data_local/test.nc/openEO.nc"
udf_file = "../00_udfs/bfast_udf.R"
save_file = "./results/local_udf_test.tif"
dimension = "t"
context = {'start_monitor': 2018}

In [25]:
# load and prepare data - depending on your data structure
dataset = xr.open_dataset(load_file, engine = 'h5netcdf', decode_coords = 'all')

In [35]:
# define runer for udf
def run(process, udf, data = None, dimension = None, context = None):
    # Run UDF executor
    t1 = time.time() # Start benchmark
    print('start: ', udf) #t1
    result = execute_udf(process, udf, data, dimension = dimension, context = context)
    t2 = time.time() # End benchmark
    print('end: ', udf) #t2

    # Print result and benchmark
    print('  Time elapsed: %s' % (t2 - t1))
    #save_result(result, save_file)

In [40]:
print('reduce_dimension bfast')
run('reduce_dimension', udf_file, data = dataset, dimension = dimension, context = context)

reduce_dimension bfast
start:  ../00_udfs/bfast_udf.R
end:  ../00_udfs/bfast_udf.R
  Time elapsed: 78.01328754425049
